# Weekly Meeting 7

* Check the context length after the preprocessing step.
* Why the ancestors are noisy or not informative for some sets of entities?
* Clarify some points about the graphs encoding.

In [1]:
from pyvis import network as net
from IPython.display import Image
from itertools import chain
import pickle as pkl
import networkx as nx
import numpy as np
import functools


with open("./knowledge_graphs/infobox_graph_depth-3.pkl", "rb") as fr:
    infobox_graph = pkl.load(fr)
    
with open("./knowledge_graphs/category_graph_depth-6_dag.pkl", "rb") as fr:
    dag = pkl.load(fr)
    
with open("./knowledge_graphs/category_graph_depth-6.pkl", "rb") as fr:
    graph = pkl.load(fr)

# DAG #
with open("./knowledge_graphs/6_lowest_common_ancestors.pkl", "rb") as fr:
    cache_ancestors = pkl.load(fr)
        
with open("./knowledge_graphs/6_lowest_common_ancestors_graph.pkl", "rb") as fr:
    cache_ancestors_graph = pkl.load(fr)    

### Context length

In the case of BART, the maximum length of the input for the encoder is 1024 subwords.

If the length of a sample is higher, it is truncated by the function "encode" of fairseq/hub_interface.py, used for encoding inputs and targets:

```python
def encode(...):
    tokens = self.bpe.encode(sentence)
    if len(tokens.split(' ')) > self.max_positions - 2:
        tokens = ' '.join(tokens.split(' ')[:self.max_positions - 2])
    ...
```

If the length of a sample is lower, there is hardly any noise in this case because Fairseq minimizes the padding within a mini-batch by grouping source and target sequences of similar length, until filling with a maximum of max_tokens tokens the mini-batch:

```python
{ 'id': tensor([ 352,  586,  403,   89, 1348, 2161,  848]), 
  'nsentences': 7, 
  'ntokens': 1798, 
  'net_input': {
      'src_tokens': tensor([[ 9157,   438, 10464,  ...,  2957,   462,     2],
        [19843,  1190, 12513,  ...,  6127,  1745,     2],
        [14009, 16086,   234,  ...,  5470,  5097,     2],
        ...,
        [    1, 33531,  9033,  ...,  1655,  3516,     2],
        [    1, 18393,  9252,  ..., 21116,   397,     2],
        [    1,   250,  4779,  ...,  7927,  1120,     2]]), 
      'src_lengths': tensor([253, 253, 253, 252, 252, 252, 252]), 
      'prev_output_tokens': tensor([[    2,  2013,  1952,  ...,     1,     1,     1],
        [    2,  8015, 37260,  ...,     1,     1,     1],
        [    2,  4656, 15862,  ...,     1,     1,     1],
        ...,
        [    2, 22098, 38187,  ...,     1,     1,     1],
        [    2, 12597, 40171,  ...,   397,     1,     1],
        [    2,  2262,  3828,  ...,  4167,  7927,  1120]])}, 
  'target': tensor([[ 2013,  1952,  1721,  ...,     1,     1,     1],
        [ 8015, 37260,  1721,  ...,     1,     1,     1],
        [ 4656, 15862,  1721,  ...,     1,     1,     1],
        ...,
        [22098, 38187,   260,  ...,     1,     1,     1],
        [12597, 40171,   260,  ...,     2,     1,     1],
        [ 2262,  3828,     9,  ...,  7927,  1120,     2]]) }
```


<br> I computed some statistics of the inputs length in different formats (after the preprocessing):

| Format | Maximum length | Average length | 75th percentile | 99th percentile |
| -- |  -- | -- | -- | -- |
| BCE | 998 | 296.61 | 323  | 876 |
| ABCE | 1018  | 321.22 | 352   | 879 |
| ACE| 440 | 135.41 | 147 | 415 |
| AE| 71 | 32.76 | 40 | 58 |

There are not truncations in any case (although one sample of ABCE is near of 1024).

### Why the ancestors are noisy for some entities?

I tried to see why the ancestors are not informative or contain noise for some entities that have gold aggregations better ranked by the BCE system than by the A.* systems (examples from the section **What confusions appear by adding the symbolic information?** of the previous meeting).

| Entities | Aggregation | Lowest Common Ancestors | Why? |
| --- | --- | --- | --- |
| (Guidant Corp., Johnson & Johnson Inc) | companies involved in the proposal | Pharmaceutical companies of the United States, Health care companies of the United States, Pharmaceutical companies, Pharmaceutical industry in the United States, Life sciences industry, Companies of the United States Boston Scientific | Contextual aggregation, the ancestors are representative for both entities but not useful for the aggregation. |
| (Carmen Ricci, William Majors) | people involved in the project | Populated places in New Mexico, Education in Canada, Towns in Alberta, People from North Rhine-Westphalia, Cities in Oklahoma, Towns in the United Kingdom | The Wikipedia searcher returns the pages for "Carmen Marc Valvo" and "William T. Major" because there aren't Wikipedia pages for the entities of the instance. The lowest common ancestor between them is "People" and, in the Wikipedia category graph, the "People" category (and other like "Politician") is used to subcategorize other categories like cities, countries, organizations, ... (figure) |
| (Michael Rips, Seydou Keita) | people involved in the project | People from Idaho, Scandals, Municipalities of Denmark, Intergovernmental organizations, Arkansas counties | Same than in the previous case. In the TESA dataset, the entity "Seydou Keïta" (photographer) is normalized as "Seydou Keita" (football player). Also, Michael Rips is not in Wikipedia and the searcher returns "Michael Ripper". Again, the lowest common ancestor between them is "People".|
| (Broad Art Foundation, Riverside Park Fund) | foundations | American people, Real estate in the United States, Construction in the United States, Arts in the United States, Urban planning, Visual arts | Broad Art Foundation does not exists in Wikipedia, and "Eli Broad" (creator of the Broad Art Foundation) is returned by the searcher. The same for Riverside Park Fund, the searcher returns the page for Riverse Park (Manhattan). The ancestors related to arts/constructions/people are because the builder of the Riverside Park appear in the category graph (Robert Moses project) and he is related with Eli Broad|
| (Marvao, Portugal) | portugal properties | Member states of the European Union, States and territories established in the 1860s, Countries in Europe, Iberian Peninsula, Southwestern European countries, Member states of the Community of Portuguese Language | There is not noise, but the ancestors are not informative. The problem here is that "Portugal" is an ancestor of "Marvao" (next cell), so the ancestors are the categories of "Portugal". Addtionally, there is useful information in the path from "Marvao" to "Portugal".|

<table><tr>
    <td> <img src="remove_by_subcat.png" alt="Drawing" style="width: 500px;"/> </td>
    <td> <img src="remove_by_supercat.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

In [2]:
entities = ("Marvao", "Portugal")
lowest_ancestors = cache_ancestors[tuple(entities)]
print(lowest_ancestors)
induced_vertices = []
for ancestor in lowest_ancestors:
    for entity in entities:
        induced_vertices.extend(nx.shortest_path(dag, ancestor, entity))

g = net.Network(notebook=True, height="500px", width="900px", directed=True)
g.show_buttons(filter_=['physics'])
g.from_nx(dag.subgraph(induced_vertices))
g.show("_.html")

['Member states of the European Union', 'States and territories established in the 860s', 'Countries in Europe', 'Iberian Peninsula', 'Southwestern European countries', 'Member states of the Community of Portuguese Language Countries']


### Clarify some points about the graph encoding

Q1: For the category graph, we don't know exactly what relationship links two nodes ("typically hyponymy or meronymy" [\[1\]](https://www.aclweb.org/anthology/W07-0201/)[\[2\]](https://www.researchgate.net/publication/295075220_Derivation_of_is_a_taxonomy_from_Wikipedia_Category_Graph)[\[3\]](https://arxiv.org/pdf/cs/0604036.pdf)) while the infobox graph is populated by many types of relationships. Should we encode in different ways both graphs?

The goal is to learn **useful** entity embeddings for the **entity aggregation task**, from a given graph $\mathcal{G} = \{(e_i, r_j, e_k)\ : \ e_i, e_k \in \mathcal{E} \wedge r_j \in \mathcal{R}\}$.

Q2: What means **useful** in our case? what information should contain the embeddings for ranking better in TESA?


I assume to use BART as neural encoder for learning entity embeddings.

Q3: to follow a denoising scheme or a discriminative approach?

Generative: $p(y_1^I | x_1^J) = \prod_{i=1}^{I} p(y_i | y_1^{i-1}, x_1^J)$
<br>Discriminative: $p(y | x_1^J) = f_{sm}(u_J^\intercal W + b)_y$

In any case, several questions:

* Q4: What is the target $y$ in our case?
     * What information should contain a target $y$ to force the model to learn **useful** entity representations? 
     
* Q5: What is the input $x$ in our case?
    * What information should remove the noise (denoising generative) or should contain the input (discriminative) to force the model to learn **useful** entity representations?
    
    

Suppose we have finetuned the BART model (depending on $y$ and $x$).

Q6: how to query the model for entity embeddings? (as they are contextual, a context is required...)


...


QF: experimentation with some structure based pretrained models? https://torchbiggraph.readthedocs.io/en/latest/pretrained_embeddings.html (~TransE + reciprocal relations trained in a Wikidata dump from 2019-03-06).


### Integrate encoders on TESA

**KG-DualAttention BART**: Given $f_{G}$, and a pre-trained BART for finetuning on TESA ($f_{T}$), combine the encoder outputs in the encoder-decoder attentions of BART. I identified two ways:

* **Early fusion**: to combine the keys and values from the encoders of $f_{G}$ and $f_{T}$ previously to use them in the encoder-decoder attention of the decoder of $f_{T}$. Basically, to compute $K_{G}\in \mathbb{R}^{T_1\times d_k}$ and $V_{G}\in \mathbb{R}^{T_1\times d_k}$ from $f^{enc}_{G}$ given an input in terms of the entities (what input for $f_{G}$?). Compute also $K_{T}\in \mathbb{R}^{T_2\times d_k}$ and $V_{T}\in \mathbb{R}^{T_2\times d_k}$ from $f^{enc}_{T}$ with the [B, C, E] input. Combine them e.g. single concatenation $K=[K_G, K_T] \in \mathbb{R}^{(T_1+T_2)\times d_k}$ and $V=[V_G, V_T] \in \mathbb{R}^{(T_1+T_2)\times d_k}$ and use $K$ and $V$ in the encoder-decoder attention layers of the decoder as usual, $Z=MultiHeadAttn(Q, K, V)$ where $Q$ is the query computed by the decoder.


* **Late fusion**: two different encoder-decoder attention layers, one attending to $f^{enc}_G$ ($K_{G}$ and $V_{G}$) and another to $f^{enc}_T$ ($K_{T}$ and $V_{T}$) are combined: **Graph-ED-Attention** ($Z_G = MultiHeadAttn(Q, K_G, V_G) \in \mathbb{R}^{T_3\times d}$) and **Instance-ED-Attention** ($Z_T = MultiHeadAttn(Q, K_T, V_T) \in \mathbb{R}^{T_3\times d}$)

# Appendix

In [3]:
from collections import defaultdict
import itertools

def get_subgraph_entities(graph, entities, depth):
    elements = defaultdict(lambda: [])
    elements[0] = entities
    for d in range(1, depth + 1):
        for element in elements[d-1]:
            elements[d].extend(list(graph.successors(element)))
    return graph.subgraph(itertools.chain(*list(elements.values())))

In [90]:
# Modification of lowest common ancestors for considering entities ancestor of other entities #
# ¿nearest common ancestor?
def is_valid_query(graph, nodes):
    return True #all([graph.has_node(node) for node in nodes])

def lowest_common_ancestors(graph, nodes, top_k=6):
    #assert nx.is_directed_acyclic_graph(graph), "Graph has to be acyclic and directed."

    def filtered_distance(term, dist):
        if term in ["Living people", "People", "Places",
                    "Use American English", "Use British English", "EngvarB"] \
                or any(list(map(lambda x: x in term.lower(), ["year of", "grey links",
                                                              "accuracy disputes", "deaths",
                                                              "births"]))):
            return float("inf")
        else:
            return dist

    if not is_valid_query(graph, nodes):
        return []

    # Ancestors modified to include also if some entity is ancestor of another entity #
    common_ancestors = list(set.intersection(*[nx.descendants(graph, node).union(set([node])) for
                                               node in nodes]))
    
    if not common_ancestors:
        return []

    sum_of_path_lengths = np.zeros((len(common_ancestors)))
    for ii, c in enumerate(common_ancestors):
        sum_of_path_lengths[ii] = functools.reduce(lambda x, y: x + y,
                                                   [nx.shortest_path_length(graph, node, c)
                                                    for node in nodes])
        sum_of_path_lengths[ii] = filtered_distance(c, sum_of_path_lengths[ii])

    indices = np.argsort(sum_of_path_lengths)[:top_k]
    
    return list(zip([common_ancestors[ii] for ii in indices],
                    [sum_of_path_lengths[ii] for ii in indices]))

In [91]:
entities = ("Joseph I. Lieberman", "Ned Lamont")
common_ancestors = lowest_common_ancestors(graph, entities, top_k=30)#cache_ancestors_graph[entities]
print(common_ancestors)
g = net.Network(notebook=True, height="500px", width="900px", directed=True)
g.show_buttons(filter_=['physics'])
g.from_nx(gr#get_subgraph_entities(graph, entities, 1))
g.show("_.html")

[('Ned Lamont', 1.0), ('Connecticut Democrats', 2.0), ('Harvard Institute of Politics', 3.0), ('Lamont family', 3.0), ('Democratic Party state governors of the United States', 3.0), ('Politicians from Washington, D.C.', 3.0), ('People from Laurel Hollow, New York', 3.0), ('Editors of Vermont newspapers', 3.0), ('Businesspeople from Washington, D.C.', 3.0), ('People from Ludlow (town), Vermont', 3.0), ('Central Connecticut State University faculty', 3.0), ('American anti–Iraq War activists', 3.0), ('People from North Haven, Maine', 3.0), ('Businesspeople from Greenwich, Connecticut', 3.0), ('Governors of Connecticut', 3.0), ('Brookings Institution', 3.0), ('State constitutional officers of Connecticut', 4.0), ('Democrats (United States)', 4.0), ('Connecticut politicians', 4.0), ('Foreign policy and strategy think tanks in the United States', 4.0), ('Democratic Party (United States) politicians', 4.0), ('Democratic Party of Connecticut', 4.0), ('Newspapers published in Vermont', 5.0), ('

### Clarify some points of the lowest common ancestors


The definition of the lowest common ancestor for two nodes is: *The LCA of v and w is the shared ancestor of v and w that is located farthest from the root (**deepest**)*

But for the experiments, I computed *the shared ancestor of v and w that is located nearest from v and w*

They are not the same because in my interpretation, the lowest common ancestor may not be the most specific category:

<img src="ancestors_properties.jpg">

The question is, what's the name of this? nearest common ancestor or ...?


---


The ancestors computed following the first definition are noisy:

(Segolene Royal, Nicolas Sarkozy)
(['Diasporas', 'Great Britain', 'States of the East Coast of the United States', 'U.S. states with multiple time zones', 'Social economy', 'Emigration'], [7.0, 6.0, 6.0, 6.0, 6.0, 6.0])


That's because almost any entity can be reached from very specific (deep) categories like "States of the East Coast of the United States":

['States of the East Coast of the United States',
 'Pennsylvania',
 'People from Pennsylvania',
 'People',
 'Living people',
 'Segolene Royal']
 
while more appropiated categories like "French politicians" appear near to the root (3.0).


---

Also, for all the experiments where I used the "lowest common ancestors", I extracted them from a DAG transformation of the category graph (computed by means of https://dl.acm.org/doi/10.1145/3091478.3091495) because the category graph has cycles and I thought that the "lowest common ancestors" were not defined for cyclic graphs (all I see in the literature is for trees or DAGs). But, the algorithm I written for computing the "lowest common ancestors" seem to work also for cyclic graphs because basically what it does is: to compute the ancestors of each entity (Dijkstra), intersect the ancestors and sort them by the sum of distances. I don't know why this is supposed not to work for cyclic graphs. 

Also, I don't know if it was a good decision to use the DAG or it's better to compute the ancestors directly on the category graph (visually, the ancestors from both graphs are very similar in terms of the useful information, and the results using the ancestors directly from the category graph seem slightly better for MAP and R@10).

*The results are obtained with the same hyper-parameters than the generative system of the paper*

| System | MAP | R@10 | MRR |
| -- |  -- | -- | -- |
| BCE | 83.07 | 93.02 | 93.90 |
| ABCE (DAG) | 83.08 | 93.11 | **94.67** |
| ABCE (CAT) | **83.19** | **93.25**  | 94.39  |

### Nearest common ancestors?

In [1]:
# Modification of lowest common ancestors for considering entities ancestor of other entities #
# ¿nearest common ancestor?
def is_valid_query(graph, nodes):
    return True #all([graph.has_node(node) for node in nodes])

def lowest_common_ancestors(graph, nodes, top_k=6):
    #assert nx.is_directed_acyclic_graph(graph), "Graph has to be acyclic and directed."

    def filtered_distance(term, dist):
        if term in ["Living people", "People", "Places",
                    "Use American English", "Use British English", "EngvarB"] \
                or any(list(map(lambda x: x in term.lower(), ["year of", "grey links",
                                                              "accuracy disputes", "deaths",
                                                              "births"]))):
            return float("inf")
        else:
            return dist

    if not is_valid_query(graph, nodes):
        return []

    # Ancestors modified to include also if some entity is ancestor of another entity #
    common_ancestors = list(set.intersection(*[nx.ancestors(graph, node).union(set([node])) for
                                               node in nodes]))
    
    if not common_ancestors:
        return []

    sum_of_path_lengths = np.zeros((len(common_ancestors)))
    for ii, c in enumerate(common_ancestors):
        sum_of_path_lengths[ii] = functools.reduce(lambda x, y: x + y,
                                                   [nx.shortest_path_length(graph, c, node)
                                                    for node in nodes])
        sum_of_path_lengths[ii] = filtered_distance(c, sum_of_path_lengths[ii])

    indices = np.argsort(sum_of_path_lengths)[:top_k]
    return sorted([common_ancestors[ii] for ii in indices])

### LCA (Wikipedia definition)

In [490]:
# Lowest common ancestors (distance to the root)

def is_valid_query(graph, nodes):
    return True #all([graph.has_node(node) for node in nodes])

def lowest_common_ancestors_root(graph, nodes, root="CategoryRoot", top_k=6):
    #assert nx.is_directed_acyclic_graph(graph), "Graph has to be acyclic and directed."

    def filtered_distance(term, dist):
        if term in ["Living people", "People", "Places",
                    "Use American English", "Use British English", "EngvarB"] \
                or any(list(map(lambda x: x in term.lower(), ["year of", "grey links",
                                                              "accuracy disputes", "deaths",
                                                              "births"]))):
            return -1
        else:
            return dist

    if not is_valid_query(graph, nodes):
        return []

    # Ancestors modified to include also if some entity is ancestor of another entity #
    common_ancestors = list(set.intersection(*[nx.ancestors(graph, node).union(set([node])) for
                                               node in nodes]))
    
    if not common_ancestors:
        return []

    sum_of_path_lengths = np.zeros((len(common_ancestors)))
    for ii, c in enumerate(common_ancestors):
        sum_of_path_lengths[ii] = nx.shortest_path_length(graph, root, c)
        sum_of_path_lengths[ii] = filtered_distance(c, sum_of_path_lengths[ii])

    indices = np.argsort(sum_of_path_lengths)[::-1][:top_k]
    return [common_ancestors[ii] for ii in indices], [sum_of_path_lengths[ii] for ii in indices]

In [491]:
with open("./knowledge_graphs/category_graph_depth-6_dag.pkl", "rb") as fr:
    dag = pkl.load(fr)

In [492]:
with open("./knowledge_graphs/category_graph_depth-6.pkl", "rb") as fr:
    graph = pkl.load(fr)

In [493]:
#aux_graph = graph.reverse()
roots = [node for node in dag.nodes if dag.in_degree(node)==0]
# add artificial root #
new_root = "CategoryRoot"
edges = [(new_root, root) for root in roots]
dag.add_edges_from(edges)
nodes = ["Marvao", "Portugal"] #["Segolene Royal", "Nicolas Sarkozy"]

In [494]:
print(lowest_common_ancestors_root(dag, nodes))

(['Diasporas', 'Human migrations', 'Major non-NATO ally', 'Great Britain', 'Human gender and sexuality', 'Physical quantities'], [7.0, 6.0, 6.0, 6.0, 6.0, 6.0])


In [485]:
nx.shortest_path_length(dag, new_root, "French politicians")

3

In [495]:
nx.shortest_path(dag, "Diasporas", "Marvao")

['Diasporas',
 'Jews and Judaism',
 'Jewish culture',
 'Israeli culture',
 'Arts in Israel',
 'Works',
 'Historiography',
 'Historiography of science',
 'Philosophy of science',
 'Epistemology of science',
 'Scientific theories',
 'Political science theories',
 'Forms of government',
 'Socialism',
 'Socialist states',
 'Portugal',
 'Geography of Portugal',
 'Populated places in Portugal',
 'Populated places in Portalegre District',
 'Marvao']

In [499]:
nx.shortest_path(dag, "Great Britain", "Portugal")

['Great Britain',
 'Scotland',
 'Scottish culture',
 'Works',
 'Historiography',
 'Historiography of science',
 'Philosophy of science',
 'Epistemology of science',
 'Scientific theories',
 'Political science theories',
 'Forms of government',
 'Socialism',
 'Socialist states',
 'Portugal']

### Pytorch BigGraph

https://github.com/facebookresearch/PyTorch-BigGraph

https://torchbiggraph.readthedocs.io/en/latest/

In [509]:
len(infobox_graph.edges)

234420

* Should it be a "natural sentence" to reduce the syntactic mismatch with respect to the pretrained model?